<a href="https://colab.research.google.com/github/U8e7s5h/portfolio/blob/master/%E9%87%8D%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%9002_%E8%BB%8A%E4%BE%A1%E6%A0%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自動車の馬力と車幅、車高から価格、ホイールベースを予想する

In [0]:
import pandas_datareader.data as pdr
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

import requests, zipfile
import io

# カリフォルニア大学からデータをダウンロード、読み込み

In [0]:
url ='http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

#ヘッダ設定
auto.columns = ['symboling','normalized-losses','make','fuel-type','aspiration','num-of-doors','body-style','drive-wheels','engine-location','wheel-base','length','width','height','curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore','stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']


# データ形式を確認

In [0]:
print('データの形式: {}'.format(auto.shape))

データの形式: (205, 26)


In [0]:
auto.head(10)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
5,2,?,audi,gas,std,two,sedan,fwd,front,99.8,177.3,66.3,53.1,2507,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,17710
7,1,?,audi,gas,std,four,wagon,fwd,front,105.8,192.7,71.4,55.7,2954,ohc,five,136,mpfi,3.19,3.40,8.5,110,5500,19,25,18920
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.40,8.3,140,5500,17,20,23875
9,0,?,audi,gas,turbo,two,hatchback,4wd,front,99.5,178.2,67.9,52.0,3053,ohc,five,131,mpfi,3.13,3.40,7.0,160,5500,16,22,?


# 「price」「horsepower」「width」「height」内の不適切データ「?」の数をカウント

In [0]:
auto = auto[['price','horsepower','width','height','wheel-base']]
auto.isin(['?']).sum()

price         4
horsepower    2
width         0
height        0
wheel-base    0
dtype: int64

# 「？」をNaNに置換して削除

In [0]:
auto = auto.replace('?', np.nan).dropna()
print('データ形式:{}'.format(auto.shape))

データ形式:(199, 5)


# もういちど「？」の数をカウント

In [0]:
auto = auto[['price','horsepower','width','height','wheel-base']]
auto.isin(['?']).sum()

price         0
horsepower    0
width         0
height        0
wheel-base    0
dtype: int64

# データ型を確認

In [0]:
print(format(auto.dtypes))

price          object
horsepower     object
width         float64
height        float64
wheel-base    float64
dtype: object


# priceとhorsepowerをobject型からint型へ変換

In [0]:
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print(format(auto.dtypes))

price           int64
horsepower      int64
width         float64
height        float64
wheel-base    float64
dtype: object


# モデルを構築し学習
model = LinearRegressionでインスタン作成、model.fitで訓練データを読み込ませて学習完了

In [0]:
# sklearnをインポート
from sklearn.model_selection import train_test_split

# 重回帰モデル用ライブラリ
from sklearn.linear_model import LinearRegression

X = auto.drop('price', axis=1)
y = auto['price']

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 重回帰　初期化と学習
model = LinearRegression()
model.fit(X_train, y_train)


# 決定係数
print('決定係数(train):{:.3f}'.format(model.score(X_train, y_train)))
print('決定係数(test):{:.3f}'.format(model.score(X_train, y_train)))

# 回帰係数と切片
print('\n回帰係数(傾き)\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('切片:{:.3f}'.format(model.intercept_))

決定係数(train):0.742
決定係数(test):0.742

回帰係数(傾き)
horsepower      78.569035
width         1318.162210
height         -21.696290
wheel-base     272.285128
dtype: float64
切片:-107854.535


trainとtestに差がなく過学習でないことがわかる